# Text analysis of restaurant reviews
## Find hamburgers!

In this notebook we analyze the Iens restaurant reviews for a city and a specific date.

In case gbq is not working you might need to update the python api client library:

```bash
sudo pip install --upgrade google-api-python-client
```

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq as gbq 
import json
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# hide warnings. `gbq.read_gbq()` gives some
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pprint import pprint

In [3]:
# project specifics
PRIVATE_KEY = '../google-credentials/gsdk-credentials.json'
PROJECT_ID = json.load(open(PRIVATE_KEY))['project_id']

In [4]:
# dataset specifics
city = 'amsterdam'
date = '20180123'
bq_table = '_'.join(['iens.iens_comments', city, date])

In [5]:
!bq ls



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

  datasetId  
 ----------- 
  iens       
  tinkering  


In [6]:
!bq ls iens


              tableId                Type    Labels   Time Partitioning  
 ---------------------------------- ------- -------- ------------------- 
  iens_amsterdam_20171107            TABLE                               
  iens_amsterdam_20171201            TABLE                               
  iens_amsterdam_20171228            TABLE                               
  iens_amsterdam_20180123            TABLE                               
  iens_amsterdam_20180124            TABLE                               
  iens_comments_amsterdam_20171228   TABLE                               
  iens_comments_amsterdam_20180123   TABLE                               
  iens_comments_diemen_20171227      TABLE                               
  iens_diemen_20171201               TABLE                               
  iens_diemen_20171227               TABLE                               
  iens_dongen_20180124               TABLE                               


In [7]:
# dataset specifics (restaurants info)
bq_table_restaurants = '_'.join(['iens.iens', city, date])

## Reading from BigQuery

To load a BigQuery table into a Pandas dataframe, all you need is a query, the project_id, and a way to authenticate.

In [8]:
query = "SELECT * FROM {}".format(bq_table)

df = gbq.read_gbq(query, project_id=PROJECT_ID, private_key=PRIVATE_KEY)

Requesting query... ok.
Job ID: job_869TiF3yIvm0Uqxi2DRbVgR3-B6U
Query running...
Query done.
Processed: 46.4 MB
Standard price: $0.00 USD

Retrieving results...
  Got page: 1; 18% done. Elapsed 13.42 s.
  Got page: 2; 42% done. Elapsed 20.39 s.
  Got page: 3; 68% done. Elapsed 26.13 s.
  Got page: 4; 86% done. Elapsed 31.92 s.
  Got page: 5; 100% done. Elapsed 36.14 s.
Got 94854 rows.

Total time taken 37.59 s.
Finished at 2018-01-26 10:00:02.


In [9]:
df.shape

(94854, 10)

# Analysis

In [10]:
df['date'] = pd.to_datetime(df['date'])
df['yearmonth'] = df['date'].dt.strftime('%Y-%m')
df['year'] = df['date'].dt.strftime('%y')
df['month'] = df['date'].dt.strftime('%m')

In [11]:
df.head().T

,0,1,2,3,4
name,Brasserie Van Dam,Kapitein Zeppos,Kapitein Zeppos,Kapitein Zeppos,Kapitein Zeppos
reserved_online,False,False,False,False,False
id,219761,220017,220017,220017,220017
rating_service,7,8,8,8,7
date,2015-07-08 00:00:00,2015-09-10 00:00:00,2012-09-25 00:00:00,2014-02-18 00:00:00,2013-07-06 00:00:00
rating_food,9,8,8,8,8
rating_decor,8,9,9,9,10
reviewer,Ben K.,Chili C.,Peter S.,Mart S.,peter1979 .
rating,8.3,8.3,8.3,8.3,8.3
comment,"Vooraf Vitello tonnato gegeten, was een verras...",Voor ons bezoek aan de Kleine Komedie nog even...,"Tja de ervaringen hieronder wisselen nogal, en...",alsje het niet weet loop je er zo aan voorbij...,Leuk restaurantje in een steegje met een heel ...


In [18]:
df.rating_food.unique()

array([ 9,  8, 10,  7,  1,  2,  4,  3,  5,  6])

# Elastic search

To run Elasticsearch as a service:
+ `brew tap homebrew/services`
+ `brew services start elasticsearch`

You can check that it's up-and-running by examining the logs:
+ `tail -n 15 /usr/local/var/log/elasticsearch.log`


In [19]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [20]:
es.ping()

True

In [21]:
res = es.cat.health(v=True, h=['timestamp', 'cluster', 'status', 'node.total'])
print(res)

timestamp cluster            status node.total
10:15:26  elasticsearch_roel yellow          1



In [79]:
print(es.cat.indices(v=True))

health status index         uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   iens_comments U46WzTepTeuJm7Wnro1TXw   5   1      94854            0    178.7mb        178.7mb



In [85]:
es.indices.delete(index='iens_comments', ignore=404)

{'acknowledged': True}

In [86]:
hyphens_and_apostrophes_strip = {
    "hyphens_and_apostrophes_strip": {
        "type": "mapping",
        "mappings": [ 
            "- => ' '",
            "' => "
        ]
    }
}

In [87]:
dutch_stop = {
    "dutch_stop": {
        "type": "stop",
        "stopwords": "_dutch_"
    }
}

In [88]:
ngram_tokenizer = {
    "ngram_tokenizer": {
        "type": "ngram",
        "min_gram": 2,
        "max_gram": 3
    }
}

In [89]:
analyzer = {
    "iens_comments_analyzer": {
        "type":           "custom",
        "char_filter":  [ "hyphens_and_apostrophes_strip" ],
        "tokenizer":      "ngram_tokenizer", #"standard",
        "filter":       [ "lowercase", "dutch_stop", "asciifolding" ]
    }
}   

In [90]:
es.indices.create(index='iens_comments', ignore=400)

es.indices.close(index='iens_comments')

es.indices.put_settings(
    index='iens_comments', 
    body={
        "analysis": {
            "char_filter": hyphens_and_apostrophes_strip,
            "tokenizer": ngram_tokenizer,
            "filter": dutch_stop,
            "analyzer": analyzer
        }
    }
)

es.indices.open(index='iens_comments')

{'acknowledged': True, 'shards_acknowledged': True}

In [91]:
es.indices.put_mapping(
    index='iens_comments',
    update_all_types=True,
    doc_type='iens_review',
    body={
        "properties": {
            "comment": {
                "type": "text",
                "analyzer": "iens_comments_analyzer"
            },
            "name": {
                "type": "keyword"            
            },
            "rating": {
                "type": "integer"
            }
        }
    }
)

{'acknowledged': True}

In [92]:
pprint(
    es.indices.get_mapping(index='iens_comments')['iens_comments']['mappings']
)

{'iens_review': {'properties': {'comment': {'analyzer': 'iens_comments_analyzer',
                                            'type': 'text'},
                                'name': {'type': 'keyword'},
                                'rating': {'type': 'integer'}}}}


In [93]:
print(es.cat.indices(v=True))

health status index         uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   iens_comments ZVNOEkiJQXaq2ysH3OdorA   5   1          0            0      1.1kb          1.1kb



In [95]:
from elasticsearch.helpers import parallel_bulk
def generate_actions(n_comments):    
    for i in range(n_comments):
        src = {
            'comment': df.loc[i, 'comment'],
            'name': df.loc[i, 'name'],
            'rating': int(df.loc[i, 'rating_food'])  
        }

        yield {
            '_op_type': 'index',
            '_source': src 
        }

In [96]:
for success, info in parallel_bulk(
    client=es, 
    actions=generate_actions(len(df)),
    index='iens_comments', 
    doc_type='iens_review',
    thread_count=4):
    if not success: print('Document insertion failed', info)

In [121]:
print(es.cat.indices(v=True))

health status index         uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   iens_comments ZVNOEkiJQXaq2ysH3OdorA   5   1      94854            0    180.9mb        180.9mb



In [99]:
res = es.search(index='iens_comments')
for review in res['hits']['hits']:
    pprint(review['_source'])
    print()

{'comment': 'Als Amsterdammer kom ik al een geruime tijd bij Brasserie '
            'Harkema. De sfeer en het decor zijn modern, maar erg gezellig. De '
            'service ervaar ik als goed en het personeel heeft kennis van de '
            'wijn en de kaart. Sinds de kaartwijziging die onlangs heeft '
            'plaatsgevonden heeft de kwaliteit een flinke boost gekregen. De '
            'keuze is wat diverser en met name de Cote de boeuf die bereid '
            'wordt in een Josper oven is aan te raden. Deze manier van '
            'bereiden bracht  een fantastische smaak met zich mee. Al met al, '
            'een grote aanrader!',
 'name': 'Brasserie Harkema',
 'rating': 9}

{'comment': 'Afgelopen donderdag met een gezelschap van 15 bij HCK gegeten. '
            'Wij hadden ivm de grootte van de groep gekozen voor het 4 gangen '
            'lounge-menu to share. Ik kan er kort over zijn: het was een '
            'perfecte keuze; fantastische gerechten, ontzettend lekker

In [109]:
res = es.search(
    index='iens_comments', 
    doc_type='iens_review',
    body={
        "query": {
            "match": {
                "comment": "burger"
            }
        }
    }
)

In [110]:
for document in res['hits']['hits']:
    print('Score: ', document['_score'])
    pprint(document['_source'], indent=2)

Score:  21.81572
{ 'comment': 'Een burger gedaan samen met mijn 5jarige zoon. Serveerster '
             'stelde kinder burger voor. Ik bestelde een gewoon '
             'classic/original met als verzoek geen uien en geen augurk.  '
             'Burgers werden gebracht. En ja hoor vol met uien en augurken '
             'zowel als topping als in de saus vies klef broodje erg zoet van '
             'smaakt burger uitgedroogd. Me zoon een burger zo klein dat hij '
             'zelfs zei papa is dit alles. Friet die er bij zat overladen met '
             'zout en eea kruide. Al met bleek einde van de €33 1burger met '
             'friet 1 cola 1 miniscule burger met friet en appelsap voor die '
             'kwaliteit. Geen burger bitch maar burger bagger',
  'name': 'Burger Bitch',
  'rating': 2}
Score:  21.747906
{ 'comment': "Wij zijn met z'n vieren gaan eten bij de Burgermeester. We "
             'werden hartelijk ontvangen en kozen de Spaanse burger, '
             'cheeseburg

## TODO: kijk hoeveel % van reviews iets over hamburgers zegt

## TODO: rating samenvoegen met score voor burger

In [122]:
len(res['hits']['hits'])

10